# 🎭 AI Companion: Universal Roleplay Bridge (Threaded)

This notebook acts as a remote 'brain' for your AI Companion. It allows you to run high-end 8B models (like Stheno-v3.2) on Google's T4 GPUs and tunnel the response back to your local machine via Ngrok.

### 🛠️ Setup Instructions:
1. **GPU Acceleration**: Go to `Runtime` > `Change runtime type` and ensure **T4 GPU** is selected.
2. **Colab Secrets (IMPORTANT)**: 
   - Click the **Key icon** (Secrets) on the left sidebar.
   - Add a new secret named `HF_TOKEN` with your [HuggingFace Token](https://huggingface.co/settings/tokens).
   - Add a new secret named `NGROK_TOKEN` with your [Ngrok Authtoken](https://dashboard.ngrok.com/get-started/your-authtoken).
   - Toggle **'Notebook access'** to ON for both.
3. **Run All**: Press `Ctrl + F9` or go to `Runtime` > `Run all`.

### 🔗 Connecting to the Local App:
1. Wait for the final cell to display the **🚀 BRIDGE ONLINE!** message.
2. Copy the **URL** (it will look like `https://xxxx-xx-xx-xx.ngrok-free.app`).
3. Open your local `settings.json` and paste the URL into `remote_llm_url`:
   ```json
   "remote_llm_url": "https://your-ngrok-url.ngrok-free.app"
   ```
4. Restart your local `main.py` script. The companion will now use the Colab GPU for all thinking!

In [ ]:
from google.colab import userdata

In [ ]:
# @title 1. Install Dependencies
!pip install -q -U fastapi uvicorn pyngrok nest_asyncio requests==2.32.4
!pip install -q -U transformers accelerate bitsandbytes torch

In [ ]:
# @title 2. Load Roleplay Specialist (Stheno-v3.2)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from threading import Thread

# --- AUTH ---
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    print("❌ ERROR: HF_TOKEN not found in Secrets!")
    HF_TOKEN = None
# ------------

model_id = "Sao10K/L3-8B-Stheno-v3.2"

print(f"Loading {model_id}... This may take a few minutes.")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=HF_TOKEN
)

print(f"\n✅ Roleplay Specialist LOADED on {torch.cuda.get_device_name(0)}!")

In [ ]:
# @title 3. Start API Server & Tunnel
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import uvicorn, nest_asyncio, re, os, time, random
from pyngrok import ngrok
from pydantic import BaseModel
from typing import List
from threading import Thread
from transformers import TextIteratorStreamer

try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
except:
    print("❌ ERROR: NGROK_TOKEN not found in Secrets!")
    NGROK_TOKEN = None

app = FastAPI()
nest_asyncio.apply()

class Message(BaseModel):
    role: str
    content: str

class ChatRequest(BaseModel):
    messages: List[Message]
    max_tokens: int = 1024
    temperature: float = 0.8

@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    chat = [{"role": m.role, "content": m.content} for m in request.messages]
    
    model_inputs = tokenizer.apply_chat_template(
        chat,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True
    ).to(model.device)

    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = {
        **model_inputs,
        "streamer": streamer,
        "max_new_tokens": request.max_tokens,
        "temperature": request.temperature,
        "do_sample": True,
        "top_p": 0.9,
    }

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    def stream_generator():
        for new_text in streamer:
            yield new_text

    return StreamingResponse(stream_generator(), media_type="text/plain")

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

ngrok.kill()

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="error")

server_thread = Thread(target=run_server)
server_thread.daemon = True
server_thread.start()

time.sleep(2)

if server_thread.is_alive():
    try:
        public_url = ngrok.connect(8000).public_url
        print("="*50)
        print(f"\n🚀 BRIDGE ONLINE!\n")
        print(f"Copy this URL to your settings.json -> remote_llm_url:")
        print(f"{public_url}\n")
        print("="*50)
    except Exception as e:
        print(f"❌ NGROK ERROR: {e}")
else:
    print("❌ SERVER ERROR: Failed to start FastAPI.")

try:
    while True: time.sleep(1)
except KeyboardInterrupt:
    print("Bridge stopped.")